In [1]:
import pandas as pd

In [13]:
metadata = pd.read_csv("./data/metadata.csv")
meta_filter = metadata[["Channel", "Category"]]
meta_filter = meta_filter.drop_duplicates()
meta_dict = meta_filter.set_index("Channel")["Category"].to_dict()

In [14]:
print(meta_dict)

{'3Blue1Brown': 'Science', 'A&E': 'News', 'About To Eat': 'Food', 'Abroad in Japan': 'Blog', 'Austin Evans': 'Tech', 'Barely Sociable': 'Informative', 'BBC News': 'News', 'Bon Appétit': 'Food', 'Brooklyn Nine-Nine': 'Comedy,Entertainment', 'BuzzFeedVideo': 'Entertainment', 'Captain Disillusion': 'Informative', 'Car Throttle': 'Automobile', 'carwow': 'Automobile', 'CDawgVA': 'Blog', 'ChrisFix': 'Automobile', 'CNET': 'Informative', 'Coder Coder': 'Tech,Informative', 'Doctor Who': 'Entertainment', 'Donut Media': 'Automobile', 'Dr. Becky': 'Science', 'DramaAlert': 'Blog', 'Drew Gooden': 'Blog', 'DRIVETRIBE': 'Automobile,Comedy', 'Eater': 'Food', 'ElectroBOOM': 'Science', 'EpicNameBro': 'VideoGames', 'Epicurious': 'Food', 'Fireship': 'Tech', 'First We Feast': 'Food,Entertainment', 'FoodTribe': 'Food', 'gameranx': 'VideoGames', 'Gordon Ramsay': 'Food', 'Hardware Canucks': 'Tech', "Hell's Kitchen": 'Food', 'Incognito Mode': 'Blog,Comedy', 'Insider News': 'News', 'Internet Historian': 'Comedy,